# SST2

In [12]:
import pandas as pd
import textflint
# load GLUE sst2 dataset and save splits as .csv
from datasets import load_dataset
dataset = load_dataset("glue", "sst2")
# rename "sentence" column to "x"
# rename "label" column to "y"
# replace 0 with negative and 1 with positive
dataset['train'] = dataset['train'].rename_column('sentence', 'x').rename_column('label', 'y')
dataset['validation'] = dataset['validation'].rename_column('sentence', 'x').rename_column('label', 'y')
dataset['test'] = dataset['test'].rename_column('sentence', 'x').rename_column('label', 'y')

def transform_labels(example):
    example["y"] = "negative" if example["y"] == 0 else "positive"
    return example

dataset['train'].to_csv('../data/GLUE_textflint/sst2_train.csv')
dataset['validation'].to_csv('../data/GLUE_textflint/sst2_val.csv')
dataset['test'].to_csv('../data/GLUE_textflint/sst2_test.csv')

Creating CSV from Arrow format:   0%|          | 0/68 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

204793

now run textflint on the dataset  
`textflint --dataset sst2_val.csv --config config.json`  
config sth like  
```
{
  "task": "SA",
  "out_dir": "./DATA/",
  "trans_methods": [
    ["SpellingError", "Typos"]
  ]
}
```

In [19]:
def transform_back(file_path, out_path):
    import json
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            # Strip whitespace and parse the JSON object
            d = json.loads(line.strip())
            data.append(d)
            
    import pandas as pd
    df = pd.DataFrame(data)
    
    # rename "x" column to "sentence"
    # rename "y" column to "label"
    df = df.rename(columns={"x": "sentence", "y": "label"})
    df.to_csv(out_path)  # "../data/GLUE_textflint/sst2_dev_textflint.csv"

In [21]:
transform_back('/Users/anna/Documents/git projects.nosync/StyleTokenizer/data/GLUE_textflint/DATA_test/trans_Pipeline_SpellingError_Typos_random_1813.json', '../data/GLUE_textflint/sst2_test_textflint.csv')

In [22]:
transform_back('/Users/anna/Documents/git projects.nosync/StyleTokenizer/data/GLUE_textflint/DATA_train/trans_Pipeline_SpellingError_Typos_random_59223.json', '../data/GLUE_textflint/sst2_train_textflint.csv')

# QNLI

In [40]:
from datasets import load_dataset
dataset = load_dataset("glue", "qnli")
from datasets import Features, Value
# Cast the label column to a string feature
for split in dataset.keys():
    dataset[split] = dataset[split].cast(
        Features({**dataset[split].features, "label": Value("string")})
    )
    dataset[split] = dataset[split].map(
        lambda x: {"label": "entailment" if x["label"] == "0" else "neutral" if x["label"] == "1" else x["label"]}
    )
    # expects 'NLI': ['premise', 'hypothesis'], y instead of question, sentence, label
    dataset[split] = dataset[split].rename_column('question', 'premise').rename_column('sentence', 'hypothesis').rename_column('label', 'y')

    # remoe elements where y is -1
    dataset[split] = dataset[split].filter(lambda example: example["y"] != -1)

dataset['train'].to_csv('../data/GLUE_textflint/qnli_train.csv')
dataset['validation'].to_csv('../data/GLUE_textflint/qnli_val.csv')
dataset['test'].to_csv('../data/GLUE_textflint/qnli_test.csv')


Filter:   0%|          | 0/5463 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5463 [00:00<?, ? examples/s]

Creating CSV from Arrow format:   0%|          | 0/105 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

1328272

In [34]:
pd.DataFrame(dataset['validation'])

,premise,hypothesis,y,idx
0,What came into force after the new constitutio...,"As of that day, the new constitution heralding...",0,0
1,What is the first major city in the stream of ...,The most important tributaries in this area ar...,1,1
2,What is the minimum required if you want to te...,In most provinces a second Bachelor's Degree s...,1,2
3,How was Temüjin kept imprisoned by the Tayichi...,The Tayichi'ud enslaved Temüjin (reportedly wi...,0,3
4,"What did Herr Gott, dich loben wir become know...","He paraphrased the Te Deum as ""Herr Gott, dich...",1,4
...,...,...,...,...
5458,How many people lived in Warsaw in 1939?,Unfortunately this belief still lives on in Po...,1,5458
5459,One of FIS' agenda items was to force women to...,Taking advantage of economic failure and unpop...,0,5459
5460,Dendritic cells are named that because they re...,Dendritic cells serve as a link between the bo...,1,5460
5461,When did it retract to a inland formation?,"However, the rainforest still managed to thriv...",1,5461


In [25]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'sentence', 'label', 'idx'],
        num_rows: 104743
    })
    validation: Dataset({
        features: ['question', 'sentence', 'label', 'idx'],
        num_rows: 5463
    })
    test: Dataset({
        features: ['question', 'sentence', 'label', 'idx'],
        num_rows: 5463
    })
})